In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_horizVert_wavelet_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,13341.90,-0.03,0.14,0.00,0.0,-0.01,0.00
3,-0.03,3662.39,-0.03,0.02,0.0,0.00,-0.00
4,0.14,-0.03,861.39,0.01,-0.0,0.00,-0.00
5,0.00,0.02,0.01,160.93,-0.0,-0.00,0.00
6,0.00,0.00,-0.00,-0.00,24.1,0.00,0.00
7,-0.01,0.00,0.00,-0.00,0.0,3.01,-0.00
8,0.00,-0.00,-0.00,0.00,0.0,-0.00,0.25


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,-0.00000,0.00004,0.00000,0.00000,-0.00003,0.00000
3,-0.00000,1.00000,-0.00001,0.00003,0.00000,0.00005,-0.00002
4,0.00004,-0.00001,1.00000,0.00003,-0.00001,0.00002,-0.00000
5,0.00000,0.00003,0.00003,1.00000,-0.00001,-0.00000,0.00001
6,0.00000,0.00000,-0.00001,-0.00001,1.00000,0.00002,0.00003
7,-0.00003,0.00005,0.00002,-0.00000,0.00002,1.00000,-0.00001
8,0.00000,-0.00002,-0.00000,0.00001,0.00003,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.21624057705787111

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.54897976e+07 1.86333114e+06 9.89170708e+04 3.23815049e+03
 6.46264989e+01 7.57005056e-01 1.24900694e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998728,-0.049258,-0.010608,-0.001975,-0.000296,-0.000038,-0.000003
1,0.048648,0.997514,-0.050203,-0.008770,-0.001311,-0.000162,-0.000014
2,0.012920,0.049134,0.997494,-0.048736,-0.006990,-0.000871,-0.000072
3,0.003005,0.010960,0.047791,0.997456,-0.051280,-0.006216,-0.000512
4,0.000598,0.002180,0.009275,0.050371,0.996654,-0.063449,-0.005110
5,0.000112,0.000406,0.001731,0.009276,0.062407,0.994574,-0.082701
6,0.000019,0.000068,0.000287,0.001537,0.010262,0.082208,0.996561


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00127227, 0.0024859 , 0.00250611, 0.00254406, 0.00334572,
       0.00542637, 0.00343884])